RentCar está desarrollando una política de reemplazo para su flotilla de automóviles en
un horizonte de planeaci ́on de 4 años. Al inicio de cada a ̃no, un automóvil se reemplaza o
se conserva en operación durante un año más. Un automóvil debe estar en servicio de 1 a 3
años. La siguiente tabla proporciona el costo de reemplazo como una función del año en que
se adquiere un automóvil y los años en operación.

In [1]:
import pandas as pd
import numpy as np

In [2]:
costs = {0: [4000, 5400, 9800], 1: [4300, 6200, 8700], 2: [4800, 7100, np.nan], 3: [4900, np.nan, np.nan]}
columns = [1, 2, 3]

actions = [
    (start_year, start_year + columns[end_year], cost)
    for start_year, cost_list in costs.items()
    for end_year, cost in enumerate(cost_list)
    if not pd.isnull(cost)
]

actions = pd.DataFrame(actions, columns=['start_year', 'end_year', 'cost'])

In [3]:
ants = 4  # Número de hormigas
epochs = 100  # Número de iteraciones
alpha = 1.0  # Importancia de las feromonas
beta = 1.0  # Importancia de la heurística
evaporation_rate = 0.1  # Tasa de evaporación de las feromonas 
pheromone_init = 0.1  # Valor inicial de feromonas
lr = 1  # Tasa de aprendizaje

In [4]:
# Iteracion 1
actions['convenience'] = 1 / actions['cost']
actions['pheromone_level'] = pheromone_init

start_years = actions['start_year'].unique()
grouped_actions = actions.groupby('start_year')

actions['proportion'] = (actions['convenience'] ** beta) * (actions['pheromone_level'] ** alpha)
actions['total'] = grouped_actions['proportion'].transform('sum')
actions['probability'] = actions['proportion'] / actions['total']
actions['cumulative'] = grouped_actions['probability'].transform('cumsum')


In [5]:
# Itearación 1
actions

,start_year,end_year,cost,convenience,pheromone_level,proportion,total,probability,cumulative
0,0,1,4000,0.000250,0.1,0.000025,0.000054,0.465353,0.465353
1,0,2,5400,0.000185,0.1,0.000019,0.000054,0.344706,0.810060
2,0,3,9800,0.000102,0.1,0.000010,0.000054,0.189940,1.000000
3,1,2,4300,0.000233,0.1,0.000023,0.000051,0.457080,0.457080
4,1,3,6200,0.000161,0.1,0.000016,0.000051,0.317007,0.774087
5,1,4,8700,0.000115,0.1,0.000011,0.000051,0.225913,1.000000
6,2,3,4800,0.000208,0.1,0.000021,0.000035,0.596639,0.596639
7,2,4,7100,0.000141,0.1,0.000014,0.000035,0.403361,1.000000
8,3,4,4900,0.000204,0.1,0.000020,0.000020,1.000000,1.000000


In [6]:
# Colonia de hormiga
for epoch in range(epochs):
    paths = []  # Guardamos los caminos de todas las hormigas
    # Aqui comienza la iteración de las hormigas
    for ant in range(ants):
        # Hormiga comienza aqui
        year = 0
        path = []
        path_cost = 0
        while True:
            available_actions = actions[actions['start_year'] == year]
            if available_actions.empty:
                break

            rand_num = np.random.rand()
            selected_action = available_actions[available_actions['cumulative'] >= rand_num].iloc[0]
            
            next_year = selected_action['end_year']
            path.append((year, next_year))
            year = next_year

            path_cost += selected_action['cost']
        
        # Hormiga Termina aqui
        paths.append([path, path_cost])
        
        # Imprime resultados de cada hormiga
        print(f'Ant {ant + 1} - Path: {path}, Cost: {path_cost}')

    # Aqui termina la iteración de las hormigas
    
    # Caminos que tomaron las hormigas
    # print(paths)
    
    actions['pheromone_level'] *= (1 - evaporation_rate) # Evaporamos las feromona
    for path, path_cost in paths:
        for start_year, end_year in path:
            actions.loc[(actions['start_year'] == start_year) & (actions['end_year'] == end_year), 'pheromone_level'] += lr / path_cost

    actions['proportion'] = (actions['convenience'] ** beta) * (actions['pheromone_level'] ** alpha)

    grouped_actions = actions.groupby('start_year')

    actions['total'] = grouped_actions['proportion'].transform('sum')
    actions['probability'] = actions['proportion'] / actions['total']
    actions['cumulative'] = grouped_actions['probability'].transform('cumsum')

    # Best path
    best_path = min(paths, key=lambda x: x[1])
    print(f'Epoch {epoch + 1}: {best_path}')
    print(f'Best path: {best_path[0]}')
    print(f'Best cost: {best_path[1]}')
    print("---------------------------------")

Ant 1 - Path: [(0, 1.0), (1.0, 3.0), (3.0, 4.0)], Cost: 15100.0
Ant 2 - Path: [(0, 2.0), (2.0, 4.0)], Cost: 12500.0
Ant 3 - Path: [(0, 2.0), (2.0, 4.0)], Cost: 12500.0
Ant 4 - Path: [(0, 1.0), (1.0, 2.0), (2.0, 4.0)], Cost: 15400.0
Epoch 1: [[(0, 2.0), (2.0, 4.0)], 12500.0]
Best path: [(0, 2.0), (2.0, 4.0)]
Best cost: 12500.0
---------------------------------
Ant 1 - Path: [(0, 2.0), (2.0, 3.0), (3.0, 4.0)], Cost: 15100.0
Ant 2 - Path: [(0, 1.0), (1.0, 4.0)], Cost: 12700.0
Ant 3 - Path: [(0, 1.0), (1.0, 2.0), (2.0, 3.0), (3.0, 4.0)], Cost: 18000.0
Ant 4 - Path: [(0, 1.0), (1.0, 2.0), (2.0, 4.0)], Cost: 15400.0
Epoch 2: [[(0, 1.0), (1.0, 4.0)], 12700.0]
Best path: [(0, 1.0), (1.0, 4.0)]
Best cost: 12700.0
---------------------------------
Ant 1 - Path: [(0, 3.0), (3.0, 4.0)], Cost: 14700.0
Ant 2 - Path: [(0, 1.0), (1.0, 3.0), (3.0, 4.0)], Cost: 15100.0
Ant 3 - Path: [(0, 2.0), (2.0, 4.0)], Cost: 12500.0
Ant 4 - Path: [(0, 2.0), (2.0, 3.0), (3.0, 4.0)], Cost: 15100.0
Epoch 3: [[(0, 2.0),

In [7]:
# Iteración final
actions

,start_year,end_year,cost,convenience,pheromone_level,proportion,total,probability,cumulative
0,0,1,4000,0.000250,0.001336,3.339956e-07,5.480422e-07,0.609434,0.609434
1,0,2,5400,0.000185,0.001045,1.934990e-07,5.480422e-07,0.353073,0.962507
2,0,3,9800,0.000102,0.000201,2.054768e-08,5.480422e-07,0.037493,1.000000
3,1,2,4300,0.000233,0.000980,2.279526e-07,2.812670e-07,0.810449,0.810449
4,1,3,6200,0.000161,0.000255,4.109491e-08,2.812670e-07,0.146106,0.956555
5,1,4,8700,0.000115,0.000106,1.221952e-08,2.812670e-07,0.043445,1.000000
6,2,3,4800,0.000208,0.001492,3.108249e-07,3.859138e-07,0.805426,0.805426
7,2,4,7100,0.000141,0.000533,7.508889e-08,3.859138e-07,0.194574,1.000000
8,3,4,4900,0.000204,0.001943,3.964904e-07,3.964904e-07,1.000000,1.000000
